In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import os

In [2]:
root_path = 'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')
exp_details = exp_details.drop(index=0)  # remove pilot experiment
#exps_to_remove = [20, 23]
#exp_details = exp_details.drop(exp_details[exp_details.Experiment.isin(exps_to_remove)].index)
exp_details

,Experiment,Larvae_present,Colony,P_yellow,C_yellow,P_red,C_red
1,11,yes,A,2,1,1,3
2,12,no,A,2,1,1,3
3,13,no,A,2,1,1,3
4,16,no,B,2,1,1,3
5,17,no,B,2,1,1,3
6,18,yes,A,2,1,1,3
7,19,yes,A,2,1,1,3
8,20,yes,A,2,1,1,3
9,21,no,B,2,1,1,3
10,22,no,B,2,1,1,6


In [3]:
def get_exp_path(exp_num, root_path):
    folderlist = os.listdir(root_path)
    exp_folder = [x for x in folderlist if x.startswith('experiment'+str(exp_num))]
    return root_path + sep + exp_folder[0]

In [4]:
def get_PC_ratios_dict(exp_details,row_idx):
    PC_ratios_dict = {}
    for color in ['yellow', 'red']:
        PC_ratios_dict[color] = [exp_details.loc[row_idx,'P_'+color], exp_details.loc[row_idx,'C_'+color]]
    return PC_ratios_dict

In [5]:
def get_crop_data_dict(exp_num, crop_file_name=r"clean_crops_transparency_corrected.csv"):
    exp_path = get_exp_path(exp_num, root_path)
    crop_data = pd.read_csv(exp_path + sep + crop_file_name, header=[0,1])
    conversion_factors = pd.read_csv(exp_path + sep + r"conversion_factors_by_weight_and_feeding_sum.csv")
    
    crop_data_dict = {}
    for color in ['red','yellow']:
        crop_data_dict[color] = crop_data.xs(color, axis=1, level=1, drop_level=False)/conversion_factors[color][0]
    
    return crop_data_dict

In [6]:
def get_ant_yellow_fraction(crop_data_dict,ant):
    y = crop_data_dict['yellow'][ant].values
    r = crop_data_dict['red'][ant].values
    return y/(y+r)

In [7]:
def get_all_yellow_fractions(crop_data_dict):
    ants = crop_data_dict['red'].columns.droplevel(1).to_list()
    yellow_fractions = pd.DataFrame(columns=ants)
    for ant in ants:
        yellow_fractions[ant] = get_ant_yellow_fraction(crop_data_dict,ant).flatten()
    return yellow_fractions

In [8]:
def get_colony_ratio(crop_data_dict,frame=-100):
    red_total = crop_data_dict['red'].iloc[frame].sum()
    yellow_total = crop_data_dict['yellow'].iloc[frame].sum()
    colony_ratio = yellow_total/(yellow_total+red_total)
    return colony_ratio

In [9]:
def plot_crop_distribution(yellow_fractions, colony_ratio, color, ax, frame=-100, hist_bins=np.linspace(0,1,11)):
    ax.hist(yellow_fractions.iloc[frame], hist_bins, color=color, alpha=0.5, density=True)
    ax.axvline(colony_ratio, linestyle='--', color='k')
    #plt.xlabel('Fraction of protein-rich food in crop')
    #plt.ylabel('PDF')

In [10]:
larvae_present = exp_details.Experiment[exp_details.Larvae_present == 'yes'].to_list()
no_larvae = exp_details.Experiment[(exp_details.Colony == 'B') & (exp_details.C_red==3)].to_list()
extra_sugar = exp_details.Experiment[exp_details.C_red==6].to_list()

In [12]:
%matplotlib qt
fig,ax = plt.subplots(2,4)
counter=0
for exp_num in larvae_present:
    crop_data_dict = get_crop_data_dict(exp_num)
    yellow_fractions = get_all_yellow_fractions(crop_data_dict)
    colony_ratio = get_colony_ratio(crop_data_dict)
    plot_crop_distribution(yellow_fractions, colony_ratio, 'b',ax[0,counter])
    counter+=1

counter=0
for exp_num in no_larvae:
    crop_data_dict = get_crop_data_dict(exp_num)
    yellow_fractions = get_all_yellow_fractions(crop_data_dict)
    colony_ratio = get_colony_ratio(crop_data_dict)
    plot_crop_distribution(yellow_fractions, colony_ratio,'m',ax[1,counter])
    counter+=1

fig.supxlabel('Fraction of protein-rich food in crop')
# fig.supylabel('PDF')
ax[0,0].set_ylabel('PDF')
ax[1,0].set_ylabel('PDF')

C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


Text(0, 0.5, 'PDF')

In [ ]:
crop_data_dict['red'].iloc[-100].sum()